In [1]:
import time
import threading
import random
import enum
import cv2
import numpy as np

In [25]:
import jetson.inference
import jetson.utils
net = jetson.inference.detectNet(argv=["--model=path_cleaner.onnx", "--labels=labels.txt", "--input-blob=input_0", "--output-cvg=scores", "--output-bbox=boxes", "--confidence=0.2"], threshold=0.1)

In [3]:
# The max driving speed of the robot
robot_speed = 0.0
horizon_offset = -0.15
lane_offset_offset = 0
missing_lane_offset = 0.2

###############################################################################
#  Overlay font and colors
###############################################################################


font = cv2.FONT_HERSHEY_SIMPLEX
color_overlay = (0, 255, 255)
color_left_lane = (255, 0, 0)
color_right_lane = (0, 0, 255)
color_center_lane = (255, 0, 255)
color_smooth_points = (255, 255, 0)

line_thickness = 5
circle_rad = 20
overlay_text_spacing = 30


###############################################################################
#  Lane detection
###############################################################################


# The reduction in resoultion for the image used for lane detection
det_res_reduction = 8

# The HSV color range for detecting the lanes (min values, max values)

#lane_color_range = (np.array([0, 0, 140]), np.array([180, 20, 255])) # White
lane_color_range = (np.array([0, 0, 180]), np.array([180, 40, 255])) # White

erode_kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,1))
dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
canny_low_threshold = 200
canny_high_threshold = 400

# The size of the sliding window for geting the smoothed stearing points
sliding_window_size = 10


###############################################################################
#  Target detection
###############################################################################


path_class = "path"
target_class = "wrapper"

# The HSV color range for detecting a picked up target (min values, max values)
target_color_range = (np.array([0, 80, 0]), np.array([30, 255, 255])) # Yellow/orange

# The desired position on the screen for a target to be able to pick it up
desired_tagret_pos = np.array([0.45, 0.68])

# The maximum error in relation to the desired tagret position before attempting to pick up the target
tagret_pos_tol = 0.05

# The maximum distance in screen space to a target before driving to it to pick it up
# 0.0 = no targets are picked up
# 0.5 = targets on the bottom half of the screen are picked up
# 1.0 = all targets are picked up
max_target_dist = 0.5


###############################################################################
#  Servos
###############################################################################


# Servo ids
class Servo(enum.Enum):
    Rotation = 1
    LowerArm = 2
    UpperArm = 3
    Claw = 4
    Camera = 5 
all_servo_ids = [s.value for s in Servo]

# The min and max angle of each servo
servo_angle_limit = {}
servo_angle_limit[Servo.Rotation.value] = (-90, 90)
servo_angle_limit[Servo.LowerArm.value] = (-90, 90)
servo_angle_limit[Servo.UpperArm.value] = (-60, 90)
servo_angle_limit[Servo.Claw.value]     = (-80,  0)
servo_angle_limit[Servo.Camera.value]   = (-40, 25)

# The initial angle each servo is set to at starup
servo_angle_init = [-10, 0, 0, 0, 0, 25]
arm_pos_init = (150, -50)

In [4]:
class RobotState(enum.Enum):
    Idle         = 0
    Detecting    = 1
    Seeking      = 2
    PickingUp    = 3
    Disposing    = 4
    PickUpFailed = 5
    
state = RobotState.Idle

In [5]:
def clamp_signed(value, min_value, max_value):
    # Returns the value clamed between the min value and max value keeping the sign of the original value.
    return np.sign(value) * np.clip(abs(value), abs(min_value), abs(max_value))

def crop_image(image, xmin, xmax, ymin, ymax):
    # Returns the cropped rectangular region defined by the parameters.
    # The parameters should be in the range (0, 1).
    h, w, _ = image.shape
    ymin = int(h * ymin)
    ymax = int(h * ymax)
    xmin = int(w * xmin)
    xmax = int(w * xmax)
    return image[ymin:ymax, xmin:xmax]

class SlidingWindow:
    # A sliding window implimentation for calculating the mean of the most recent N values.
    def __init__(self, size):
        self.values = []
        self.size = size

    def add(self, value):
        self.values.append(value)
        if len(self.values) > self.size:
            self.values.pop(0)
            
    def mean(self):
        return np.median(self.values)


# The position of the next displayed text
display_text_y = 0

def display_text(overlay, text):
    # Displays the text on the overlay stacked from the top left of the screen to the bottom
    global display_text_y
    display_text_y += overlay_text_spacing
    cv2.putText(overlay, text, (0, display_text_y), font, 1.0, color_overlay, 2, cv2.LINE_AA)
    
def display_text_reset():
    # Resets the position of the next displayed text, called each frame
    global display_text_y
    display_text_y = 0

In [6]:
from jetbot import Camera, bgr8_to_jpeg

import ipywidgets
from IPython.display import display

camera_width = 1640
camera_height = 1232

image_widget_scale = 0.5

widget_width = int(camera_width * image_widget_scale)
widget_height = int(camera_height * image_widget_scale)

image_widget = ipywidgets.Image(width = widget_width, height = widget_height)
display(image_widget)

display_frame = None
camera_lock = threading.Lock()
display_lock = threading.Lock()

def process_image(frame):
    return frame

def process_thread_func():
    global process_thread_running
    while process_thread_running:
        time.sleep(0.01)
        with camera_lock:
            if camera.value is None:
                continue
            frame = camera.value.copy()
        
        try:
            display_text_reset()
            frame = process_image(frame)
            frame = cv2.resize(frame, (widget_width, widget_height))
            with display_lock:
                display_frame = bgr8_to_jpeg(frame)
                image_widget.value = display_frame
                               
        except Exception as e:
            process_thread_running = False
            camera.stop()
            raise e

Image(value=b'', height='616', width='820')

In [7]:
# Init connection to servos and robot
from SCSCtrl import TTLServo
from jetbot import Robot
robot = Robot()

Succeeded to open the port
Succeeded to change the baudrate


In [8]:
# Limit the frequency of servo commands to avoid communication errors
servoCtrlTime = 0.001

# The number of steps to rotate the servos 1 degree
steps_per_deg = (TTLServo.servoInputRange/TTLServo.servoAngleRange)

# The current target value of each servo
servo_angle_current = servo_angle_init.copy()

def move_servo(servo, angle, speed=150):
    # Moves the servo to the given angle
    # Returns the estimated time needed for the move to complete using the given speed
    
    if type(servo) is Servo:
        servo = int(servo.value)
        
    angle = int(np.clip(angle, servo_angle_limit[servo][0], servo_angle_limit[servo][1]))
    
    offset = angle - servo_angle_current[servo]
    servo_angle_current[servo] = angle
    
    move_time = abs(offset*steps_per_deg) / speed
    
    TTLServo.servoAngleCtrl(servo, angle, 1, speed)
    time.sleep(servoCtrlTime)
    return move_time

def move_servo_t(servo, angle, dt):
    # Moves the servo to the given angle
    # Returns the speed used for the move to complete in the given time
    
    if type(servo) is Servo:
        servo = int(servo.value)
        
    angle = int(np.clip(angle, servo_angle_limit[servo][0], servo_angle_limit[servo][1]))
    
    offset = angle - servo_angle_current[servo]
    servo_angle_current[servo] = angle
    
    speed = abs(offset*steps_per_deg) / dt
    
    TTLServo.servoAngleCtrl(servo, angle, 1, int(speed))
    time.sleep(servoCtrlTime)
    return speed
    
def reset_servo(servo, speed=150):
    # Moves the servo to the initial position
    # Returns the estimated time needed for the move to complete using the given speed
    if type(servo) is Servo:
        servo = int(servo.value)
    return move_servo(servo, 0, speed)
    
def set_arm_pos(x, y):
    # Moves the arm to the given position
    la, ua = TTLServo.xyInput(x, y)
    servo_angle_current[Servo.LowerArm.value] = int(la+90)
    servo_angle_current[Servo.UpperArm.value] = int(-ua)
    time.sleep(servoCtrlTime)
    
def set_arm_pos_t(x, y, dt):
    # Moves the arm to the given position over the given duration
    la, ua = TTLServo.xyInputSmooth(x, y, dt)
    servo_angle_current[Servo.LowerArm.value] = int(la+90)
    servo_angle_current[Servo.UpperArm.value] = int(-ua)
    time.sleep(servoCtrlTime)

# Set all servos to the init angle
for s in all_servo_ids:
    move_servo(s, servo_angle_init[s])

if not arm_pos_init is None:
    set_arm_pos(arm_pos_init[0], arm_pos_init[1])

In [9]:
def apply_detection_mask(image, detections, ignore_detections, scale):
    """Masks regions of interest"""
    
    # Remove any regions outside of detections
    mask = np.zeros_like(image)
    for d in detections:
        det_rect = np.array([[
            (d.Left//scale, d.Top//scale),
            (d.Right//scale, d.Top//scale),
            (d.Right//scale, d.Bottom//scale),
            (d.Left//scale, d.Bottom//scale),
        ]], np.int32)
        cv2.fillPoly(mask, det_rect, 255)
    
    # Remove any regions inside of ignore_detections
    for d in ignore_detections:
        det_rect = np.array([[
            (d.Left//scale, d.Top//scale),
            (d.Right//scale, d.Top//scale),
            (d.Right//scale, d.Bottom//scale),
            (d.Left//scale, d.Bottom//scale),
        ]], np.int32)
        cv2.fillPoly(mask, det_rect, 0)
        
    # Remove any regions in the top half of the image
    height, width = image.shape
    polygon = np.array([[
        (0, 0),
        (width, 0),
        (width, height / 2),
        (0, height / 2),
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 0)
    
    return cv2.bitwise_and(image, mask)

def detect_line_segments(edges):
    """Breakes the edge detected image into line segments"""
    
    # Distance precision in pixel, i.e. 1 pixel
    rho = 1  
    # Angular precision in radian, i.e. 1 degree
    angle = np.pi / 180
    # Minimal of votes
    min_threshold = 10  
    line_segments = cv2.HoughLinesP(edges, rho, angle, min_threshold, np.array([]), minLineLength=10, maxLineGap=15)
    return line_segments

def average_slope_intercept(image, line_segments, xmin, xmax, slope_dir):
    """
    Calculates the average slope and intercept of multiple line segments.
    
    Parameters
    ----------
    image
        The image the line segments are from
    line_segments
        The line segments
    xmin
        [0..1] The min x value of the line segments to consider.
    xmax
        [0..1] The max x value of the line segments to consider.
    slope_dir
        1 or -1 to indicate the direction of the slope we want
    """
    
    if line_segments is None:
        return None

    height, width, _ = image.shape
    xmin, xmax = xmin*width, xmax*width
    fits = []
    
    for line_segment in line_segments:
        for x1, y1, x2, y2 in line_segment:
            if x1 == x2:
                # Horizontal line
                continue
            if x1 <= xmax and x2 <= xmax and x1 >= xmin and x2 >= xmin:
                if y2 < y1:
                    y1, y2 = y2, y1
                    x1, x2 = x2, x1
                fit = np.polyfit((x1, x2), (y1, y2), 1)
                slope = fit[0]
                intercept = fit[1]
                if slope*slope_dir > 0:
                    fits.append((slope, intercept))
    
    fit_average = np.average(fits, axis=0) if len(fits) > 0 else None
    return fit_average

def get_intersect(rs, ri, ls, li):
    '''
    Calculates the intersection point of the two lines
    
    Parameters
    ----------
    rs & ls
        The line slopes
    ri & li
        The line intercepts
    '''
    
    xi = (ri-li) / (ls-rs+0.01)
    yi = rs * xi + ri
    return xi, yi

def draw_lines(overlay, lines):
    """Draws the given lines on the overlay"""
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                color = (random.randrange(255), random.randrange(255), random.randrange(255))
                cv2.line(overlay, (x1*det_res_reduction, y1*det_res_reduction), (x2*det_res_reduction, y2*det_res_reduction), color, line_thickness)

def get_line(image, slope, intercept):
    """Calculates the start and end points of the line stretching the width of the image"""
    x0 = 0
    x1 = image.shape[1]
    
    y0 = int(intercept + slope*x0)
    y1 = int(intercept + slope*x1)
    
    return ((x0, y0), (x1, y1))

def get_angle(p0, p1, p2):
    """Calcualtes the angle between the lines p0p1 and p1p2"""
    v0 = np.array(p0) - np.array(p1)
    v1 = np.array(p2) - np.array(p1)
    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
    if angle < 0:
        angle += np.pi
    return angle

# Sliding windows for smoothing the c_x0 and c_xmax values
c_x0_window = SlidingWindow(sliding_window_size)
c_xmax_window = SlidingWindow(sliding_window_size)

def lane_detect(image, overlay, detections, ignore_detections):
    """
    Calculates the steering point in the horizon and the offset in the lane
    
    Parameters
    ----------
    image
        The image to find the lanes on
    overlay
        The image text and lines are drawn on
    detections
        The detected lanes
    ignore_detections
        Regions we want to ignore
    """
    
    # Process image
    resized = cv2.resize(image, (image.shape[1]//det_res_reduction, image.shape[0]//det_res_reduction))
    
    hsv = cv2.cvtColor(resized, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lane_color_range[0], lane_color_range[1])
    
    mask = cv2.erode(mask, erode_kernel, iterations=2)
    mask = cv2.dilate(mask, dilate_kernel, iterations=1)
    
    edges = cv2.Canny(mask, canny_low_threshold, canny_high_threshold)
    
    edges = apply_detection_mask(edges, detections, ignore_detections, det_res_reduction)
    
    # Get lines
    line_segments = detect_line_segments(edges)
    
    # Draw the found line segments
    draw_lines(overlay, line_segments)
    
    step = 0.2
    l, r = None, None
    stop = 0.7
    for x in np.arange(0, 1, step):
        if x >= stop:
            break
        if l is None:
            l = average_slope_intercept(resized, line_segments, 0, x, -1)
        if r is None:
            r = average_slope_intercept(resized, line_segments, 1-x, 1, 1)
        if (not l is None or not r is None) and stop == 1:
            stop = 1-x
        if not l is None and not r is None:
            break
    display_text(overlay, f'Stop: {stop}')
    
    if l is None and r is None:
        return image, 0, 0, 0
    
    rs, ri = r if not r is None else (0, 0)
    ls, li = l if not l is None else (0, 0)
    ri, li = ri*det_res_reduction, li*det_res_reduction
    
    l_line = get_line(image, ls, li)
    r_line = get_line(image, rs, ri)
    
    # Calculate center lane
    xi, yi = get_intersect(rs, ri, ls, li)
    angr = get_angle((1000000, yi), (xi, yi), r_line[1])
    angl = get_angle((1000000, yi), (xi, yi), l_line[1])
    angc = (angl+angr) / 2
    
    slope = np.sin(angc)/np.cos(angc)
    if r is None or l is None:
        slope = -slope
    
    c_y0 = image.shape[0]
    c_x0 = xi + (c_y0 / slope) if not slope == 0 else 0
    c_x0 = np.clip(c_x0, -1000000, 1000000)
    
    c_ymax = 0
    c_xmax = xi + (c_ymax / slope) if not slope == 0 else 0
    c_xmax = np.clip(c_xmax, -1000000, 1000000)
    
    # Calculate lane offset
    c_x0_window.add(c_x0)
    mean_c_x0 = c_x0_window.mean()
    lane_offset = 2*mean_c_x0 / image.shape[1] - 1
    
    # Calculate horizon
    c_xmax_window.add(c_xmax)
    mean_c_xmax = c_xmax_window.mean()
    horizon = 2*mean_c_xmax / image.shape[1] - 1
    
    # Draw the lanes
    cv2.line(overlay, l_line[0], l_line[1], color_left_lane, line_thickness)
    cv2.line(overlay, r_line[0], r_line[1], color_right_lane, line_thickness)
    
    # Draw center lines
    cv2.circle(overlay, (int(c_x0), int(c_y0)), radius=circle_rad, color=color_center_lane, thickness=-1)
    cv2.circle(overlay, (int(c_xmax), int(c_ymax)), radius=circle_rad, color=color_center_lane, thickness=-1)
    cv2.line(overlay, (int(c_xmax), int(c_ymax)), (int(c_x0), int(c_y0)), color_center_lane, line_thickness)
    
    cv2.circle(overlay, (int(mean_c_x0), c_y0), radius=circle_rad, color=color_smooth_points, thickness=-1)
    cv2.circle(overlay, (int(mean_c_xmax), 0), radius=circle_rad, color=color_smooth_points, thickness=-1)
    cv2.line(overlay, (int(mean_c_xmax), 0), (int(mean_c_x0), c_y0), color_smooth_points, line_thickness)
    
    mlo = missing_lane_offset if r is None else (-missing_lane_offset if l is None else 0)
    display_text(overlay, f'MLO: {mlo:.2f}')
    return image, horizon + horizon_offset + mlo, lane_offset + lane_offset_offset+mlo*4, 1
    

In [10]:
def go_to_target(overlay, pos, desired_pos, speed):
    delta = desired_pos - pos
    
    l, r = 0, 0
    
    r += delta[0]
    l -= delta[0]
        
    r += delta[1]
    l += delta[1]
    
    display_text(overlay, f'Motor power: ({delta[0]:.2f}, {delta[1]:.2f})')
    
    if abs(l) < tagret_pos_tol and abs(r) < tagret_pos_tol:
        robot.stop()
        return True
    
    l, r = clamp_signed(l*speed*2, 0.15, 1), clamp_signed(r*speed*2, 0.15, 1)
    robot.set_motors(l, r)
    return False

def ywait(t):
    start_time = time.time()
    while (time.time() - start_time) < t:
        yield False
    
def grab_target(center):
    ypos = np.interp(center[1], [0.65, 0.70], [160, 140])
    
    set_arm_pos_t(int(ypos), -160, 1)
    yield from ywait(1)
    
    move_servo_t(Servo.Claw, -80, 1.5)
    set_arm_pos_t(int(ypos), -150, 1.5)
    yield from ywait(1.5)
        
    set_arm_pos_t(150, -60, 1)
    yield from ywait(1)
    yield True
    
def failed_grab():
    dt = move_servo(Servo.Claw, 0, 300)
    yield from ywait(dt)
    yield True

def drop_target():
    dt = move_servo(Servo.Rotation, -90, 300)
    yield from ywait(dt)
    
    set_arm_pos_t(150, -150, 1)
    yield from ywait(1)
    
    dt = move_servo(Servo.Claw, 0, 300)
    yield from ywait(dt)
    
    set_arm_pos_t(150, -50, 1)
    yield from ywait(1)
    
    dt = reset_servo(Servo.Rotation, 300)
    yield from ywait(dt)
    yield True

grab_gen = None
drop_gen = drop_target()
failed_grab_gen = failed_grab()

In [11]:
target_pos = np.zeros(2)
target_index = -1

def detect(image):
    image_cuda = jetson.utils.cudaFromNumpy(image)
    detections = net.Detect(image_cuda)
    return detections
    
def drive(overlay, horizon, offset, speed):   
    l = np.interp(horizon, [-2, -1, -0.5, 0, 1], [-0.5, 0.2, 0.5, 0.8, 1])
    r = np.interp(horizon, [-1, 0, 0.5, 1, 2], [1, 0.8, 0.5, 0.2, -0.5])
    
    offset_coef = 1
    
    l = l + np.interp(offset, [-1, 0, 1], [-0.5, 0.1, 1]) * offset_coef
    r = r + np.interp(offset, [-1, 0, 1], [1, 0.1, -0.5]) * offset_coef
    
    m = max(abs(l), abs(r))
    l, r = l/m, r/m
    
    display_text(overlay, f'Motor balance: ({l:.2f}, {r:.2f})')
    
    l, r = clamp_signed(l*speed, 0.1, 1), clamp_signed(r*speed, 0.1, 1)
    robot.set_motors(l, r)
    
def has_grabbed_target(image):
    crop = crop_image(image, 0.38, 0.56, 0, 0.2)
    hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, target_color_range[0], target_color_range[1])
    mask = cv2.dilate(mask, dilate_kernel, iterations=10)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]

    if len(cnts) == 0:
        return False
    
    c = max(cnts, key=cv2.contourArea)
    area = cv2.contourArea(c)
    return area > 5000

def process_image(image):
    global state, target_pos, grab_gen, drop_gen, failed_grab_gen, target_index
    
    isize = (image.shape[1], image.shape[0])
    
    overlay = np.zeros_like(image)
    display_text(overlay, state.name)

    if state is RobotState.Idle:
        robot.stop()
        reset_servo(Servo.Rotation, 150)
    
    if state is RobotState.Detecting or state is RobotState.Seeking:
        detections = detect(image)
        for d in detections:
            class_name = net.GetClassDesc(d.ClassID)
            left = int(np.clip(d.Left, 0, isize[0]))
            right = int(np.clip(d.Right, 0, isize[0]))
            top = int(np.clip(d.Top, 0, isize[1]))
            bottom = int(np.clip(d.Bottom, 0, isize[1]))
            cv2.rectangle(overlay, (left, top), (right, bottom), color_overlay, 1)
            cv2.putText(overlay, f'{class_name} {(d.Confidence*100):.1f}%', (left, top-40), font, 1.0, color_overlay, 2, cv2.LINE_AA)
            cv2.putText(overlay, f'({(d.Center[0]/isize[0]):.2f}, {(d.Center[1]/isize[1]):.2f}) {d.Area:.0f}', (left, top-5), font, 1.0, color_overlay, 2, cv2.LINE_AA)
        target_detections = [d for d in detections if net.GetClassDesc(d.ClassID) == target_class]
        path_detections = [d for d in detections if net.GetClassDesc(d.ClassID) == path_class]
        
        target_index = -1
        if len(target_detections) > 0:
            index, target = max(enumerate(target_detections), key=lambda d: d[1].Bottom)
            if target.Bottom >= int(isize[1] * (1-max_target_dist)):
                target_index = index
                display_text(overlay, f'{target_index}, {target_detections[target_index].Bottom}')
    else:
        target_detections = []
        path_detections = []
    
    if state is RobotState.Detecting:
        image, horizon, lane_offset, speed = lane_detect(image, overlay, path_detections, target_detections)
        display_text(overlay, f'Horizon: {horizon:.2f}')
        display_text(overlay, f'Lane offset: {lane_offset:.2f}')
        drive(overlay, horizon, lane_offset, speed*robot_speed)
        
        if target_index != -1:
            state = RobotState.Seeking
    
    if state is RobotState.Seeking:
        if target_index != -1:
            d = target_detections[target_index]
            target_pos = np.array([d.Center[0] / isize[0], d.Center[1] / isize[1]])
            at_target = go_to_target(overlay, target_pos, desired_tagret_pos, robot_speed)
            if at_target:
                state = RobotState.PickingUp
                pass
        else:
            state = RobotState.Detecting
            
    if state is RobotState.PickingUp:    
        if grab_gen is None:
            grab_gen = grab_target(target_pos)
        if next(grab_gen):
            grab_gen = None
            if has_grabbed_target(image):
                state = RobotState.Disposing
            else:
                state = RobotState.PickUpFailed
    
    if state is RobotState.PickUpFailed:
        if next(failed_grab_gen):
            failed_grab_gen = failed_grab()
            state = RobotState.Detecting
    
    if state is RobotState.Disposing:
        if next(drop_gen):
            drop_gen = drop_target()
            state = RobotState.Detecting
    
    if image.shape != overlay.shape:
        image = cv2.resize(image, (overlay.shape[1], overlay.shape[0]))
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    return cv2.addWeighted(image, 1, overlay, 1, 1)

Switch states

In [12]:
state = RobotState.Detecting
speed = 0.3

In [23]:
state = RobotState.Detecting
speed = 0.0

In [14]:
state = RobotState.Idle
speed = 0.0
robot.stop()

Manual control

In [15]:
robot.forward(0.5)

In [16]:
robot.backward(0.5)

In [17]:
robot.left(0.5)

In [18]:
robot.stop()

Start / Stop

In [24]:
# (Re)start the camera and the image processing thread

try:
    camera.stop()
except:
    pass

try:
    process_thread_running = False
    if process_thread.isAlive():
        process_thread.join()
except:
    pass

camera = Camera.instance(width=camera_width, height=camera_height)
process_thread_running = True
process_thread = threading.Thread(target=process_thread_func, daemon=True)
process_thread.start()

In [ ]:
# Stop the camera and the image processing thread
process_thread_running = False
if process_thread.isAlive():
    process_thread.join()
camera.stop()